### How to connect to Postgres

> create docker volume and run the services:
<br>`docker volume create --name pgdatax` <br>
`docker-compose up -d` <br>

> to take it down<br>
> `docker-compose down`


> <b>pgAdmin UI</b> is at localhost:5050 with login: `jamesbond@007.com` | password: `goldeneye` <br>
  New Server > General: name `any_name` > Connection: hostname `postgres`,  username: `jamesbond`, password: `goldeneye` 

### Pandas & Postgres DB Connections

In [1]:
from typing import Optional
from dotenv import dotenv_values
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# configuration to local Postgres.
config = dotenv_values("../containers/postgres/postgres.env")
HOST, PORT = "127.0.0.1", 5432  
user_name = config.get("POSTGRES_USER", "postgres")
user_password = config.get("POSTGRES_PASSWORD", "admin")

In [3]:
# connection string to our service
CONNECTION_STRING = f'postgresql://{user_name}:{user_password}@{HOST}:{PORT}'

In [4]:
class Engine:

    """context managing engine
    Args:
        connection_string: string uri to postgres services
        database: databas name. default to None
    """

    def __init__(self, connection_string: str, database: Optional[str] = None) -> None:
        self.connection_string = connection_string
        self.database = database

    def __enter__(self) -> create_engine:
        if self.database is not None:
            self.connection_string = f"{self.connection_string}/{self.database}"

        self.engine = create_engine(self.connection_string)
        return self.engine

    def __exit__(self, type, value, traceback) -> None:
        self.engine.dispose()

In [5]:
# executing raw SQL query e.g. drop and create database
DB_NAME = "weapons"
with Engine(CONNECTION_STRING) as engine:
    autoengine = engine.execution_options(isolation_level="AUTOCOMMIT")
    
    with autoengine.connect() as conn:
        conn.execute(f"DROP DATABASE IF EXISTS {DB_NAME};")
        conn.execute(f"CREATE DATABASE {DB_NAME};")

In [7]:
# sending dataframe to database weapons, replacing if it exists
# create fake data in theme of 007
dataf = pd.DataFrame({
                    "bullets":["kolibri", "bonecrusher", "tranter"], 
                    "damage":[.8, .76, .98],
                    "date":pd.date_range(start="18/03/2021", periods=3)
                    })

TABLE_NAME = "bullets"
with Engine(CONNECTION_STRING, database=DB_NAME) as conn:
   # pandas awesomeness
    dataf.to_sql(TABLE_NAME, conn, 
                 if_exists="replace", 
                 index=False)

In [10]:
# read data from postgres
query = """
        SELECT 
            bullets
            ,damage
            ,"date" AS creation_date
        FROM "bullets" 
        WHERE damage >= 0.78
        """
with Engine(CONNECTION_STRING, database=DB_NAME) as conn:
    result = pd.read_sql_query(query, conn)

result

,bullets,damage,creation_date
0,kolibri,0.80,2021-03-18
1,tranter,0.98,2021-03-20


In [11]:
result.dtypes

bullets                  object
damage                  float64
creation_date    datetime64[ns]
dtype: object

In [13]:
# cleaning up ;)
with Engine(CONNECTION_STRING) as engine:
    autoengine = engine.execution_options(
                            isolation_level="AUTOCOMMIT"
    )
    with autoengine.connect() as conn:
        conn.execute(f"DROP DATABASE {DB_NAME};")